In [47]:
# Recupera los nombres de todos los archivos del directorio

import os

def listar_archivos(directorio, modificar_nombre=None):
    if modificar_nombre is None:
        modificar_nombre = lambda x: x
    archivos = [archivo for archivo in os.listdir(directorio) if archivo.endswith('.pdf') or archivo.endswith('.csv')]
    archivos = [modificar_nombre(os.path.join(directorio, archivo)) if modificar_nombre else archivo for archivo in archivos]
    return archivos

In [48]:
paper_dir = listar_archivos('../papers')
paper_dir

['../papers/Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.pdf']

In [49]:
# Funcion que elimina la extencion .csv que se encuentra al final de cada archivo
def eliminar_extension(archivo):
    if archivo.endswith('.csv'):
        return archivo[:-4]
    return archivo

extracted_dir = listar_archivos('./data/raw', eliminar_extension)
extracted_dir

['./data/raw/Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.pdf']

In [50]:
# Recuperar nombres de archivos que estan en paper_dir, pero no en extracted_dir
def archivos_faltantes(paper_dir, extracted_dir):
    return [(archivo, os.path.basename(archivo)) for archivo in paper_dir if os.path.basename(archivo) not in [os.path.basename(exr_dir) for exr_dir in extracted_dir]]

new_files = archivos_faltantes(paper_dir, extracted_dir)
new_files

[]

In [51]:
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import convert_to_dataframe

for path_file, file_name in new_files:
    elements = partition_pdf(
        filename=path_file,                                     # mandatory
        strategy="hi_res",                                     # mandatory to use ``hi_res`` strategy
        extract_images_in_pdf=True,                            # mandatory to set as ``True``
        extract_image_block_types=["Image", "Table"],          # optional
        extract_image_block_to_payload=False,                  # optional
        extract_image_block_output_dir=f"./images/{file_name}/",  # optional - only works when ``extract_image_block_to_payload=False``
        )
    df = convert_to_dataframe(elements)
    df.to_csv(f"data/raw/{file_name}.csv", index=True)
    print(f"Archivo {file_name} extraido con exito")